#Extractor y Analizador de Preferencias en Redes Sociales para Ideas de Regalos

Utilizando técnicas avanzadas de análisis de datos, esta herramienta extrae y evalúa las preferencias y gustos de los usuarios en redes sociales, transformando esta información en recomendaciones de regalos personalizados. Con la finalidad de ofrecer opciones de regalos que no solo sean únicas y significativas, sino también accesibles y fáciles de adquirir, nuestro sistema aplica algoritmos inteligentes para sugerir regalos que mejor se alinean con los intereses de la persona. El resultado final es una lista curada de tres regalos ideales, acompañada de enlaces para su busqueda directa en Amazon, brindando una solución completa y eficiente para hacer de cada ocasión especial un momento inolvidable.

El código presentado es una función de Python para importar y procesar datos de un archivo CSV. La función leer_csv extrae una lista de intereses o 'seguidos' de una columna específica, basada en el nombre del archivo y la columna proporcionados por el usuario. Este proceso es fundamental para analizar las preferencias de una persona, facilitando la generación personalizada de ideas de regalos. La interacción con el usuario se realiza a través de entradas en la consola, donde se solicitan los nombres del archivo CSV, la columna relevante y la persona destinataria del regalo.

Este paquete es la biblioteca oficial proporcionada por OpenAI, que facilita la interacción con las APIs de OpenAI, incluyendo el acceso a modelos avanzados de inteligencia artificial

In [ ]:
pip install openai==0.28

Empezamos con el codigo importanto todos los recursos necesarios y solicitando los datos para poder hacer el análisis requerido

In [ ]:
import csv
import re
import openai
import os
import unicodedata

# Pedir al usuario información relevante
nombre_archivo = input("Ingresa el nombre del archivo CSV (incluyendo la extensión .csv): ")
nombre_persona = input("Ingresa el nombre de la persona a la que le vas a dar el regalo: ")
genero_persona = input("Ingresa el genero de la persona: ")
edad_persona = input("Ingresa la edad aproximada de la persona: ")
datos_persona = "La persona tiene"+ edad_persona + " años y su genero es "+ genero_persona

Ingresa el nombre del archivo CSV (incluyendo la extensión .csv): IG-Follower-Email-Scraper_1702104246421.csv
Ingresa el nombre de la persona a la que le vas a dar el regalo: Paulina
Ingresa el genero de la persona: Femenino
Ingresa la edad aproximada de la persona: 22


En esta parte solicitamos que se ingrese la clave API KEY para mantener la seguridad y personalización de la persona a usar el modelo

In [ ]:
# Pedir al usuario que ingrese la clave API
openai_api_key = input("Ingresa tu clave API de OpenAI: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Requerimos seleccionar solo la columna de usuarios y limpiarla para que la inteligencia artificial pueda trabajar de mejor manera

In [ ]:
#Lee el archivo csv, lo limita a 1000 usuarios como maximo y limpia los datos para que sean faciles de leer para la IA
def leer_y_limpiar_csv(nombre_archivo, limite=1000):
    seguidos = []
    with open(nombre_archivo, mode='r', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            if row and len(seguidos) < limite:  # Verifica si la fila no está vacía y si no se ha alcanzado el límite
                usuario = row[4]  # Asumiendo que los nombres de usuario están en la quinta columna (índice 4)
                usuario_limpio = re.sub(r'[0-9]', '', usuario)  # Eliminar números
                usuario_limpio = usuario_limpio.replace('_', ' ').replace('.', ' ')  # Reemplazar '_' y '.' con espacios
                seguidos.append(usuario_limpio)
            else:
                break  # Romper el bucle si se alcanza el límite
    return seguidos

Este código utiliza la API de OpenAI para generar recomendaciones de regalos personalizadas. Mediante un prompt detallado, solicita a ChatGPT-4 sugerir tres regalos asequibles basados en una lista de intereses, proporcionando así ideas creativas y específicas para obsequios navideños.

In [ ]:
seguidos = leer_y_limpiar_csv(nombre_archivo)
def generar_recomendaciones_de_regalos(seguidos, openai_api_key):
    openai.api_key = openai_api_key

    # Crear un prompt que incluya los seguidos y pida recomendaciones de regalos
    prompt = "Basado en los siguientes intereses y personas seguidas: {}\n".format(", ".join(seguidos))
    prompt += "Por favor, sugiere 3 regalos baratos y fáciles de encontrar para Navidad. Solo requiero titulos cortos, porque con esos titulos voy a buscar los productos en Amazon."
    prompt += "Por favor ten encuenta también los siguientes factores de genero y edad aproximada: {}\n".format(", ".join(datos_persona))

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",  # Asegúrate de que este es el modelo correcto
            messages=[
                {"role": "system", "content": "Tu prompt de sistema aquí"},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message['content'].strip().split('\n')
    except Exception as e:
        print(f"Error al generar recomendaciones: {e}")
        return []

# Guardar las recomendaciones de regalos
regalos = generar_recomendaciones_de_regalos(seguidos, openai_api_key)

Este código toma cada título de regalo en la lista regalos, elimina los acentos, lo divide en palabras y crea un enlace de búsqueda de Amazon México utilizando esas palabras. Luego imprime cada regalo junto con su respectivo enlace.

In [ ]:
#Quitar los acentos para generar correctamente los links
import unicodedata
def eliminar_acentos(texto):
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

# Generar los links de búsqueda de Amazon México
def generar_enlace_amazon(busqueda):
    # Eliminar comillas y otros caracteres no deseados
    busqueda_limpia = eliminar_acentos(busqueda.replace('"', '').replace('.', '').replace(',', ''))
    # Separar las palabras y unirlas con '+'
    palabras = busqueda_limpia.split()
    url_amazon = "https://www.amazon.com.mx/s?k=" + '+'.join(palabras)
    return url_amazon

# Guardar los enlaces
enlaces_amazon = [generar_enlace_amazon(regalo) for regalo in regalos]

# Texto introductorio para la lista de regalos
print(f"Dada la lista de seguidos que me diste de {nombre_persona}, pienso que estos regalos pueden gustarle: ")

for regalo, enlace in zip(regalos, enlaces_amazon):
    print(f"Regalo: {regalo}, Enlace de Amazon: {enlace}")


Dada la lista de seguidos que me diste de Paulina, pienso que estos regalos pueden gustarle: 
Regalo: 1. "Libro de cocina saludable", Enlace de Amazon: https://www.amazon.com.mx/s?k=1+Libro+de+cocina+saludable
Regalo: 2. "Set de joyería minimalista", Enlace de Amazon: https://www.amazon.com.mx/s?k=2+Set+de+joyeria+minimalista
Regalo: 3. "Kit de cuidado para plantas", Enlace de Amazon: https://www.amazon.com.mx/s?k=3+Kit+de+cuidado+para+plantas


Este fragmento de código define una función en Python que, utilizando la API de OpenAI, genera explicaciones personalizadas sobre por qué ciertos regalos son adecuados para una persona específica. La función crea un prompt detallado que incluye los intereses de la persona y los regalos sugeridos, y luego utiliza ChatGPT-4 para elaborar una explicación coherente y bien fundamentada.

In [ ]:
def generar_explicaciones(regalos, nombre_persona):
    # Simplificar el prompt para evitar exceder el límite de caracteres
    prompt = f"Explica por qué crees que los siguientes regalos son adecuados para {nombre_persona} dado los datos que analisaste basandote en su lista de seguidos de instagram, quien tiene intereses variados:\n"
    for regalo in regalos:
        prompt += f"- {regalo}\n"
    prompt += "Explicación:"

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message["content"]
    except Exception as e:
        print(f"Error al generar explicaciones: {e}")
        return ""

# Ejemplo de uso de la función
explicaciones = generar_explicaciones(regalos, nombre_persona)
print("La explicación para estos regalos es:\n", explicaciones)

La explicación para estos regalos es:
 1. "Libro de cocina saludable": Paulina sigue varias páginas relacionadas con el fitness y la alimentación saludable en Instagram. Este interés en la comida sana y su deseo aparente de mantener un estilo de vida saludable hacen que un libro de cocina saludable sea un regalo ideal para ella. Ella podrá encontrar nuevas recetas para apoyar su dieta y mantenerla emocionada y comprometida con su salud.

2. "Set de joyería minimalista": Al analizar a quienes sigue Paulina en Instagram, notamos que sigue a varias marcas y personas influyentes en el campo de la moda, especialmente en lo que respecta a tendencias minimalistas y elegantes. Por lo tanto, un set de joyería minimalista seguramente complementará su sentido del estilo y será muy apreciado.

3. "Kit de cuidado para plantas": Paulina sigue a algunas páginas de jardinería y plantas en Instagram, lo que sugiere que tiene interés en las plantas y posiblemente ya tenga algunas o esté interesada en em

Este fragmento de código nos ayudara a poder visualizar de una manera creativa como podrían verse los regalos sugeridos debajo del árbol navideño

In [ ]:
#Generar la imagen con Dalle3
def generar_imagen_dalle(descripcion, openai_api_key):
    openai.api_key = openai_api_key

    try:
        response = openai.Image.create(
            model="dall-e-3",  #modelo de DALL·E 3
            prompt=descripcion,
            n=1,  # Número de imágenes a generar
            size="1024x1024"  # Tamaño de la imagen
        )
        return response.data[0]['url']  # URL de la imagen generada
    except Exception as e:
        print(f"Error al generar la imagen: {e}")
        return ""

# Imprimir la imagen
descripcion = f"Una escena navideña con un árbol decorado y los tres regalos sugeridos, siendo todos visibles. Que son: {regalos}"
url_imagen = generar_imagen_dalle(descripcion, openai_api_key)
print("URL de la imagen generada:", url_imagen)


URL de la imagen generada: https://oaidalleapiprodscus.blob.core.windows.net/private/org-xiWjKf2e54NggRLrsfLh5JFQ/user-cfoGGhomQXBmh3hOlZhgp0dr/img-TQMBccsugj2dkCVokWZwTL5n.png?st=2023-12-10T21%3A03%3A31Z&se=2023-12-10T23%3A03%3A31Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-12-09T23%3A11%3A47Z&ske=2023-12-10T23%3A11%3A47Z&sks=b&skv=2021-08-06&sig=Kr8C%2Bv3yYrilVOUEEXGKRfAW/leArCoaXnfAfv7v6X0%3D


Es importante calcular los costos del resultado del proceso para generar los posibles regalos, por lo que aquí esta el código para eso

In [ ]:
# Ejemplo de cálculo de costo considerando la longitud de las respuestas de ChatGPT y una imagen de DALL-E

# Tarifas estimadas
PRECIO_CHATGPT_POR_TOKEN = 0.00002  # Precio estimado por token para ChatGPT
PRECIO_DALLE_POR_IMAGEN = 0.02      # Precio por imagen generada por DALL·E

# Estimación de tokens: Asumiendo que cada token tiene aproximadamente 4 caracteres
num_tokens_regalos = sum(len(regalo) for regalo in regalos) / 4
num_tokens_amazon = sum(len(enlace) for enlace in enlaces_amazon) / 4
num_tokens_explicacion = len("La explicación para estos regalos es: " + explicaciones) / 4

# Calcular el costo total de los tokens
costo_tokens = (num_tokens_regalos + num_tokens_amazon + num_tokens_explicacion) * PRECIO_CHATGPT_POR_TOKEN

# Calcular el costo total incluyendo una imagen de DALL·E
costo_total = costo_tokens + PRECIO_DALLE_POR_IMAGEN

print(f"El costo total en dolares americanos de esta ejecución fue de: ${costo_total:.5f}")

El costo total en dolares americanos de esta ejecución fue de: $0.02708
